# Class 21:  joint entropy and the REVEAL algorithm

We'll use the bladder cancer gene expression data to test out the REVEAL algorithm. First, we'll load the data and filter to include only genes for which the median log2 expression level is > 12.  That should give us 164 genes to work with.

In [1]:
import pandas
gene_matrix_for_network_df = pandas.read_csv("shared/bladder_cancer_genes_tcga.txt", sep="\t")
gene_matrix_for_network = gene_matrix_for_network_df.as_matrix()
print(gene_matrix_for_network.shape)

(4473, 414)


Filter the matrix to include only rows for which the column-wise median is > 12; matrix should now be 164 x 414; this is not strictly necessary but will help prevent us from over-burderning the EC2 instance and it will enable us to easily compute the partial correlation matrix using the inverse of the covariance matrix. Print the size of the filtered matrix, as a sanity check.

In [2]:
import numpy
gene_matrix_np = numpy.array(gene_matrix_for_network)
genes_keep = numpy.where(numpy.median(gene_matrix_np, axis=1) > 12)
matrix_filt = gene_matrix_np[genes_keep, ][0]
print(matrix_filt.shape)
N = matrix_filt.shape[0]
M = matrix_filt.shape[1]

(164, 414)


Binarize the gene expression matrix using the mean value as a breakpoint, turning it into a NxM matrix of logicals (true/false).  Call it `gene_matrix_binarized`. Print out the dimensions of this matrix.

In [14]:
print(matrix_filt[0][0:15])
gene_matrix_binarized = numpy.array(matrix_filt, dtype=bool)
mean = matrix_filt.mean()
print("Matrix mean:", mean)

for n in range(N):
    for m in range(M):
        gene_matrix_binarized[n][m] = matrix_filt[n][m] > mean

[ 1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  1.  1.]
Matrix mean: 0.468761046306


Check a test case for six entries:

In [17]:
print(gene_matrix_binarized[0][0:12])

[ True  True  True  True  True  True  True False False False False False]


The core part of the REVEAL algorithm is a function that can compute the joint entropy of a collection of binary (TRUE/FALSE) vectors X1, X2, ..., Xn (where length(X1) = length(Xi) = M).
Write a function `entropy_multiple_vecs` that takes as its input a nxM matrix (where n is the number of variables, i.e., genes, and M is the number of samples in which gene expression was measured). The function should use the log2 definition of the Shannon entropy. It should return the joint entropy H(X1, X2, ..., Xn) as a scalar numeric value. I have created a skeleton version of this function for you, in which you can fill in the code. I have also created some test code that you can use to test your function, below.

In [54]:
from math import log2

def entropy_multiple_vecs(binary_vecs):
    table = {}
    for i in range(len(binary_vecs[0])):
        key = ""
        for j in range(len(binary_vecs)):
            key += str(binary_vecs[j][i])
        
        table[key] = table.get(key, 0) + 1
        
    #print(table)
    #print("Sum:", sum(table.values()))
    
    for key in table:
        table[key] /= N
        
    #print(table)
    
    result = 0
    
    for key in table:
        result += table[key] * log2(table[key])
        
    return -result
    
print("3 rows:", entropy_multiple_vecs(gene_matrix_binarized[0:3,]))
print("4 rows:", entropy_multiple_vecs(gene_matrix_binarized[0:4,]))

3: 4.0828541011059665
4: 6.456595821500659


This test case should produce the value 3.938:

In [55]:
print(entropy_multiple_vecs(gene_matrix_binarized[0:3,]))

4.0828541011059665
